# Variance-level plots

### Classes and modules

In [1]:
#Lets have matplotlib "inline"
%matplotlib inline

import os
import sys

#Import packages we need
import numpy as np
import datetime
from IPython.display import display
import copy

#For plotting
import matplotlib
from matplotlib import pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable

# plt.rcParams["lines.color"] = "w"
# plt.rcParams["text.color"] = "w"
# plt.rcParams["axes.labelcolor"] = "w"
# plt.rcParams["xtick.color"] = "w"
# plt.rcParams["ytick.color"] = "w"

plt.rcParams["image.origin"] = "lower"

import pycuda.driver as cuda

GPU Ocean-modules:

In [2]:
from gpuocean.utils import IPythonMagic, Common

In [3]:
%cuda_context_handler gpu_ctx

In [4]:
ls = [6, 7, 8]

In [5]:
from gpuocean.utils import DoubleJetCase

args_list = []
init_list = []

for l in ls:
    doubleJetCase = DoubleJetCase.DoubleJetCase(gpu_ctx, DoubleJetCase.DoubleJetPerturbationType.SteadyState, ny=2**l, nx=2**(l+1))
    doubleJetCase_args, doubleJetCase_init, _ = doubleJetCase.getInitConditions()

    args_list.append(doubleJetCase_args)
    init_list.append(doubleJetCase_init)

### Read variances from file

In [6]:
# source_path = "/home/florianb/havvarsel/multilevelDA/doublejet/scripts/VarianceLevelsDA/2023-09-26T11_10_32"
source_path = "/home/florianb/havvarsel/multilevelDA/doublejet/scripts/VarianceLevelsDA/2023-09-25T13_55_37"

In [7]:
vars = np.load(source_path+"/vars_"+str(10*24*3600)+".npy")
diff_vars = np.load(source_path+"/diff_vars_"+str(10*24*3600)+".npy")

In [8]:
vars, diff_vars

(array([[1.72024918e+02, 1.04131174e+11, 1.13910620e+11],
        [9.96223389e+02, 9.19111270e+11, 1.05711672e+12],
        [2.69934961e+03, 3.53910050e+12, 3.04662079e+12]], dtype=float32),
 array([[6.9555695e+02, 6.6310321e+11, 7.5657334e+11],
        [1.6901177e+03, 2.0647869e+12, 1.4460638e+12]], dtype=float32))

### Read work from file

In [9]:
work_path = "/home/florianb/havvarsel/multilevelDA/doublejet/scripts/PracticalCost/2023-09-18T12_54_47"

In [10]:
works = np.mean(np.load(work_path+"/costsPure.npy"), axis=-1)[:-1]
diff_works = np.mean(np.load(work_path+"/costsPartnered.npy"), axis=-1)[:-1]

Use theoretical work

In [11]:
works = [1/(8**2), 1/(8), 1]
diff_works = [1/(8), 1]

In [12]:
works, diff_works

([0.015625, 0.125, 1], [0.125, 1])

## Ensembles with equal work

In [13]:
SL_Ne = 50

In [14]:
SL_Ne2 = works[-1]*SL_Ne/works[-2]
SL_Ne2

400.0

In [15]:
SL_Ne3 = works[-1]*SL_Ne/works[-3]
SL_Ne3

3200.0

In [16]:
from gpuocean.utils import MultiLevelAnalysis

In [17]:
analysis = MultiLevelAnalysis.Analysis(args_list, vars, diff_vars, works, diff_works)

In [18]:
ML_Ne = analysis.optimal_Ne(1.345e-1)
ML_Ne

array([88, 72, 40], dtype=int32)

In [19]:
analysis.work(ML_Ne)/analysis.works[-1]

50.375

In [20]:
analysis2 = MultiLevelAnalysis.Analysis(args_list[1:], vars[1:], diff_vars[1:], works[1:], diff_works[1:])

In [21]:
ML_Ne2 = analysis2.optimal_Ne(1.355e-1)
ML_Ne2

array([84, 39], dtype=int32)

In [22]:
analysis2.work(ML_Ne2)/analysis.works[-1]

49.5

### Theoretical error

In [23]:
SLerr = np.sqrt(vars[-1]/SL_Ne)
SLerr

array([7.3475842e+00, 2.6604888e+05, 2.4684492e+05], dtype=float32)

In [24]:
analysis.theoretical_error(ML_Ne)

array([7.3395014e+00, 2.4902359e+05, 2.1898402e+05], dtype=float32)

In [25]:
analysis2.theoretical_error(ML_Ne2)

array([7.4294114e+00, 2.5275492e+05, 2.2285259e+05], dtype=float32)